In [1]:
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

from convnets.utilities import utils

In [2]:
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '/home/mksnkv/Documents/classification/passport_2class_divided_small/train'
validation_data_dir = '/home/mksnkv/Documents/classification/passport_2class_divided_small/validation'

weigths_path = '/home/mksnkv/models/simple/conv_two_class_passport_weights.h5'
structure_path = '/home/mksnkv/models/simple/conv_two_class_structure.json'
nb_train_samples = 1000
nb_validation_samples = 300
epochs = 25
batch_size = 20
num_classes=2

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(224, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

train_generator, val_generator = utils.init_generators(train_data_dir, 
                                                       validation_data_dir, 
                                                       img_width, 
                                                       img_height, 
                                                       batch_size)

reduce_lr_acc = ReduceLROnPlateau(monitor='lr', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='max')
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size,
    epochs=epochs,
    callbacks = [reduce_lr_acc])

Found 1000 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 50 steps, validate for 15 steps
Epoch 1/25
 1/50 [..............................] - ETA: 1s

ValueError: logits and labels must have the same shape ((None, 2) vs (None, 1))

In [ ]:
model.save_weights(weigths_path)
utils.save_model_structure_to_json(structure_path, model)

In [ ]:
utils.plot(history)
  

